# Recommender system:
## A collaborative filtering algorithm based on non-negative matrix factorization

In [2]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

Initialization values:

In [3]:
# svd values
n_factors     = 150
n_epochs      = 20
init_mean     = 0
init_std_dev  = .1
lr            = .005
reg           = .001
verbose       = False

# crossvalidation
n_splits      = 5


Import user-item-rating set:

In [4]:
from surprise import Reader, Dataset

reader = Reader(line_format='user item rating', sep=',')

data = Dataset.load_from_file('data/train.csv', reader=reader)

Load the cythonmagic extension (for cython code use the magic function %%cython):

In [5]:
%load_ext Cython

The SVD class:

In [6]:
%%cython

cimport numpy as np
import numpy as np

class SVD:
        
    """A collaborative filtering algorithm based on Non-negative Matrix
    Factorization. Adapted from Surprise
    
    Args:
        n_factors: The number of factors. 
            Default is ``100``.
        n_epochs: The number of iteration of the SGD procedure. 
            Default is ``20``.
        reg: The regularization term :math:`\gamma`. 
            Default is ``0.06``.
        lr: The learning rate :math:`\lambda`.
            Default is ``0.005``.
        verbose: If ``True``, prints the current epoch. Default is ``False``.
        
    Attributes:
        pu(numpy array of size (n_users, n_factors)): The user factors (only
            exists if ``fit()`` has been called)
        qi(numpy array of size (n_items, n_factors)): The item factors (only
            exists if ``fit()`` has been called)
        bu(numpy array of size (n_users)): The user biases (only
            exists if ``fit()`` has been called)
        bi(numpy array of size (n_items)): The item biases (only
            exists if ``fit()`` has been called)
    """

    
    def __init__(self, n_factors=100, n_epochs=20, init_mean=0,
                 init_std_dev=.1, lr=.005, reg=.06, verbose=False):

        self.n_factors = n_factors
        self.n_epochs = n_epochs
        self.init_mean = init_mean
        self.init_std_dev = init_std_dev
        self.lr = lr
        self.reg = reg
        self.verbose = verbose
        
    def fit(self, trainset):

        self.sgd(trainset)

        return self
        
    def sgd(self, trainset):

        # user biases
        cdef np.ndarray[np.double_t] bu
        # item biases
        cdef np.ndarray[np.double_t] bi
        # user factors
        cdef np.ndarray[np.double_t, ndim=2] pu
        # item factors
        cdef np.ndarray[np.double_t, ndim=2] qi
        
        
        cdef double global_mean = trainset.global_mean
        cdef double lr = self.lr
        cdef double reg = self.reg
        
        cdef int u, i, f
        cdef double r, err, dot, puf, qif
        
        pu = np.random.normal(self.init_mean, self.init_std_dev,
                              (trainset.n_users, self.n_factors))
        qi = np.random.normal(self.init_mean, self.init_std_dev,
                              (trainset.n_users, self.n_factors))
        bu = np.zeros(trainset.n_users, np.double)
        bi = np.zeros(trainset.n_items, np.double)

        
        for current_epoch in range(self.n_epochs):
            if self.verbose:
                print("Processing epoch {}".format(current_epoch))
            for u, i, r in trainset.all_ratings():

                # compute current error
                dot = 0  # <q_i, p_u>
                for f in range(self.n_factors):
                    dot += qi[i, f] * pu[u, f]
                err = r - (global_mean + bu[u] + bi[i] + dot)

                # update biases
                bu[u] += lr * (err - reg * bu[u])
                bi[i] += lr * (err - reg * bi[i])

                # update factors
                for f in range(self.n_factors):
                    puf = pu[u, f]
                    qif = qi[i, f]
                    pu[u, f] += lr * (err * qif - reg * puf)
                    qi[i, f] += lr * (err * puf - reg * qif)

        self.bu = bu
        self.bi = bi
        self.pu = pu
        self.qi = qi
        self.trainset = trainset
        
    def test(self, testset):
        rmse = 0
        mae = 0
        
        errors = [r - self.estimate(u, i) for u, i, r in testset]
        rmse = np.sqrt(np.mean([error**2 for error in errors]))
        mae = np.mean([abs(error) for error in errors])
        
        return (rmse, mae)
        
        
    def estimate(self, raw_u, raw_i):

        known_user = raw_u in self.trainset._raw2inner_id_users
        known_item = raw_i in self.trainset._raw2inner_id_items
        
        est = self.trainset.global_mean

        if known_user:
            u = self.trainset.to_inner_uid(raw_u)
            est += self.bu[u]

        if known_item:
            i = self.trainset.to_inner_iid(raw_i)
            est += self.bi[i]

        if known_user and known_item:
            est += np.dot(self.qi[i], self.pu[u])
        
        # clip estimation to scale of ratings
        est = min(4, est)
        est = max(0, est)

        return est

Initialization of SVD class and cross-validation:

In [7]:
import time
from surprise.model_selection import KFold

algo = SVD(n_factors=n_factors, 
           n_epochs=n_epochs, 
           init_mean=init_mean, 
           init_std_dev=init_std_dev, 
           lr=lr, 
           reg=reg, 
           verbose=verbose)

# define a cross-validation iterator
kf = KFold(n_splits=n_splits)

rmse = np.zeros(n_splits)
mae = np.zeros(n_splits)
fit_time = np.zeros(n_splits)
test_time = np.zeros(n_splits)
i = 0

for trainset, testset in kf.split(data):
    # train and test algorithm.
    start_time = time.time()
    algo.fit(trainset)
    fit_time[i] = time.time()-start_time
    
    start_time = time.time()
    rmse[i], mae[i] = algo.test(testset)
    test_time[i] = time.time()-start_time
    
    i += 1

Print results:

In [8]:
from six import iteritems

print('Evaluating RMSE, MAE of algorithm SVD on {0} split(s).'.format(n_splits))
print()
row_format = '{:<18}' + '{:<8}' * (n_splits + 2)
s = row_format.format('', *['Fold {0}'.format(i + 1) for i in range(n_splits)]
                      + ['Mean'] + ['Std'])
s += '\n'
s += row_format.format('RMSE (testset)',*['{:1.4f}'.format(vals) for vals in rmse] 
                       + ['{:1.4f}'.format(np.mean(rmse))] 
                       + ['{:1.4f}'.format(np.std(rmse))])
s += '\n'
s += row_format.format('MAE (testset)',*['{:1.4f}'.format(vals) for vals in mae] 
                       + ['{:1.4f}'.format(np.mean(mae))] 
                       + ['{:1.4f}'.format(np.std(mae))])
s += '\n'
s += row_format.format('Fit time',*['{:1.4f}'.format(vals) for vals in fit_time] 
                       + ['{:1.4f}'.format(np.mean(fit_time))] 
                       + ['{:1.4f}'.format(np.std(fit_time))])
s += '\n'
s += row_format.format('Test time',*['{:1.4f}'.format(vals) for vals in test_time] 
                       + ['{:1.4f}'.format(np.mean(test_time))] 
                       + ['{:1.4f}'.format(np.std(test_time))])
print(s)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.5070  0.5104  0.5095  0.5109  0.5119  0.5099  0.0017  
MAE (testset)     0.2933  0.2946  0.2946  0.2934  0.2953  0.2942  0.0008  
Fit time          18.8010 18.6230 17.7710 21.2860 21.4920 19.5946 1.5073  
Test time         0.6800  0.6000  0.6100  0.7100  0.6400  0.6480  0.0417  


Fit model with full dataset:

In [9]:
trainset_all = data.build_full_trainset()
algo.fit(trainset_all)

Read qualifying set and predict all ratings:

In [10]:
# read qualifying set
Xq = np.genfromtxt("qualifying.csv", delimiter=",", dtype=np.int)

# create array to store predictions
predValues = np.chararray([Xq.shape[0],3], itemsize=10)

# predict ratings for each element in our qualifying set
for index, [user, item] in enumerate(Xq):
    
    pred = algo.estimate(str(user), str(item))
    
    # near-integer roundoff
    if (np.abs( pred - np.rint( pred )) <= 0.1):
        pred = np.rint(pred)
    else:
        pred = pred
        
    predValues[index] = [user, item, str(pred)]

In [11]:
# save predictions to a csv file
import csv
with open('qualifying_preds.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerows(predValues.decode('utf'))